<a href="https://colab.research.google.com/github/jadaksnyder/Artificial-Intelligence/blob/main/TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from geopy.distance import geodesic
import requests

# Store the 10 Phil locations in a dictionary
locations = {
    "Phil1": (40.94448, -78.96854),
    "Phil2": (40.94384, -78.97215),
    "Phil3": (40.94321, -78.97842),
    "Phil4": (40.94816, -78.96191),
    "Phil5": (40.94507, -78.96537),
    "Phil6": (40.94296, -78.97668),
    "Phil7": (40.95426, -78.97565),
    "Phil8": (40.94505, -78.96812),
    "Phil9": (40.94424, -78.96806),
    "Phil10": (40.93908, -78.99365),
}

def meters_to_miles(meters):
    return meters * 0.000621371

def distance(location_from, location_to):
    """Fetches driving distance between two coordinates using OSRM API"""
    url = f"http://router.project-osrm.org/route/v1/driving/{location_from[1]},{location_from[0]};{location_to[1]},{location_to[0]}?overview=false"
    response = requests.get(url)

    if response.status_code == 200:
        try:
            data = response.json()
            distance_in_meters = data["routes"][0]["legs"][0]["distance"]  # Distance in meters
            return round(meters_to_miles(distance_in_meters), 2)  # Convert to miles and round
        except KeyError:
            print("Error: Unable to extract distance.")
    else:
        print(f"Error: Failed to retrieve data. Status code: {response.status_code}")

    return float('inf')  # Return large value if API fails

# Create the distance matrix automatically
def create_distance_matrix():
    keys = list(locations.keys())
    num_locations = len(keys)
    distance_matrix = np.zeros((num_locations, num_locations))

    for i in range(num_locations):
        for j in range(num_locations):
            if i != j:
                dist = distance(locations[keys[i]], locations[keys[j]])
                if dist is not None:
                    distance_matrix[i][j] = dist  # Already rounded in `distance()`

    return distance_matrix, keys

# Implement Nearest Neighbor Heuristic
def nearest_neighbor(start_index=0, distance_matrix=None):
    """Solves the TSP using a Nearest Neighbor Heuristic"""
    unvisited = set(range(len(distance_matrix)))
    unvisited.remove(start_index)
    route = [start_index]
    current = start_index

    while unvisited:
        next_city = min(unvisited, key=lambda city: distance_matrix[current][city])
        unvisited.remove(next_city)
        route.append(next_city)
        current = next_city

    route.append(start_index)  # Return to starting city
    return route

# Run the fixed code
distance_matrix, keys = create_distance_matrix()
print("Distance Matrix:")
print(distance_matrix)

# Solve TSP using nearest neighbor starting from Phil1 (index 0)
route = nearest_neighbor(0, distance_matrix)
print("Optimal Route (Nearest Neighbor):")
print(" ➝ ".join([keys[i] for i in route]))


Distance Matrix:
[[0.   0.25 0.66 0.61 0.24 0.49 1.02 0.05 0.06 1.7 ]
 [0.25 0.   0.49 0.81 0.43 0.31 0.91 0.26 0.25 1.53]
 [0.59 0.41 0.   1.15 0.77 0.1  1.33 0.64 0.59 1.19]
 [0.59 0.78 1.2  0.   0.36 1.03 1.42 0.64 0.54 2.24]
 [0.24 0.43 0.85 0.38 0.   0.67 1.07 0.29 0.18 1.88]
 [0.49 0.31 0.1  1.05 0.67 0.   1.23 0.54 0.49 1.24]
 [1.02 0.91 1.4  1.44 1.07 1.23 0.   0.95 0.96 2.44]
 [0.05 0.42 0.84 0.66 0.28 0.67 0.95 0.   0.18 1.88]
 [0.06 0.25 0.66 0.56 0.18 0.49 0.96 0.1  0.   1.7 ]
 [1.7  1.53 1.19 2.26 1.88 1.24 2.44 1.75 1.7  0.  ]]
Optimal Route (Nearest Neighbor):
Phil1 ➝ Phil8 ➝ Phil9 ➝ Phil5 ➝ Phil4 ➝ Phil2 ➝ Phil6 ➝ Phil3 ➝ Phil10 ➝ Phil7 ➝ Phil1
